In [1]:
      
from dask.distributed import Client
import dask.array as da


import skimage as ski
import numpy as np
import dask
import glob
import platform

In [7]:

client = Client('tcp://10.2.40.2:8786')

print(client)


client


<Client: 'tcp://10.2.40.2:8786' processes=8 threads=8, memory=850.47 GiB>


<Client: 'tcp://10.2.40.2:8786' processes=8 threads=8, memory=850.47 GiB>


array = da.random.random((20000, 20000,200), chunks=(1000, 1000,100))
result = array.mean().compute() # This will be executed across your 5 PCs!

print(f"The mean of the large array is: {result}")

In [8]:
def process_image(fname):
    # If this system is on linux replace the fname
    # with the full path to the image file.
    # For example: fname = '/home/user/images/image.tif'
    if platform.system() == 'Linux':
        fname = fname.replace('\\', '/').replace('//ion', '/n')
    if platform.system() == 'Darwin':
        fname = fname.replace('\\', '/').replace('//ion', '/Volumes')
    import cellpose.models as models
    model = models.CellposeModel(gpu=True)
    img = ski.io.imread(fname)
    rslts = model.eval(img, channel_axis=None, z_axis=0, stitch_threshold=0.5)[0]
    ski.io.imsave(fname+'f', rslts)
    return 0

In [9]:
fnames = glob.glob('\\\\ion\\projects/smc/public/SMC/dask/data/*.tif')[0:10] * 5
fnames

['\\\\ion\\projects/smc/public/SMC/dask/data\\img0.tif',
 '\\\\ion\\projects/smc/public/SMC/dask/data\\img1.tif',
 '\\\\ion\\projects/smc/public/SMC/dask/data\\img2.tif',
 '\\\\ion\\projects/smc/public/SMC/dask/data\\img3.tif',
 '\\\\ion\\projects/smc/public/SMC/dask/data\\img4.tif',
 '\\\\ion\\projects/smc/public/SMC/dask/data\\img5.tif',
 '\\\\ion\\projects/smc/public/SMC/dask/data\\img6.tif',
 '\\\\ion\\projects/smc/public/SMC/dask/data\\img0.tif',
 '\\\\ion\\projects/smc/public/SMC/dask/data\\img1.tif',
 '\\\\ion\\projects/smc/public/SMC/dask/data\\img2.tif',
 '\\\\ion\\projects/smc/public/SMC/dask/data\\img3.tif',
 '\\\\ion\\projects/smc/public/SMC/dask/data\\img4.tif',
 '\\\\ion\\projects/smc/public/SMC/dask/data\\img5.tif',
 '\\\\ion\\projects/smc/public/SMC/dask/data\\img6.tif',
 '\\\\ion\\projects/smc/public/SMC/dask/data\\img0.tif',
 '\\\\ion\\projects/smc/public/SMC/dask/data\\img1.tif',
 '\\\\ion\\projects/smc/public/SMC/dask/data\\img2.tif',
 '\\\\ion\\projects/smc/public/

In [10]:
rslts = [dask.delayed(process_image)(fname) for fname in fnames]
# Compute the results in parallel
dask.compute(*rslts)

(0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0)

In [11]:
# When you're done, you can close the connection
client.close()